# Convolutional Neural Networks

Realizaremos un ejercicio descrito en el blog www.aprendemachinelearning.com para hacer reconocimiento de imágenes.

# Importar Librerías

In [1]:
import numpy as np
import os
import re
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [14]:
import tensorflow
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential,Input,Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, Convolution2D, MaxPooling2D,BatchNormalization,LeakyReLU
from tensorflow.keras.preprocessing.image import ImageDataGenerator
print(tensorflow.__version__)

2.12.0


In [49]:
data_entrenamiento = os.path.join(os.getcwd(), 'DataSet', 'train')
data_validacion = os.path.join(os.getcwd(), 'DataSet', 'test')

epocas = 10
altura, longitud = 216, 384
batch_size=32
pasos=np.ceil((1633*0.8/batch_size)-1)#100
pasos_validacion=np.ceil((810*0.2/batch_size)-1)#200
filtrosConv1=32
filtrosConv2=64
tamanio_filtro1=(3,3)
tamanio_filtro2=(2,2)
tamanio_pool=(2,2)
clases=8
lr=0.0005


In [50]:
entrenamiento_datagen = ImageDataGenerator(
    rescale=1./255
)
validacion_datagen = ImageDataGenerator(
    rescale=1./255
)

In [51]:
imagen_entrenamiento = entrenamiento_datagen.flow_from_directory(
 data_entrenamiento,
 target_size=(altura, longitud),
 batch_size = batch_size,
 class_mode = 'categorical'
)

Found 1633 images belonging to 8 classes.


In [52]:
imagen_validacion = validacion_datagen.flow_from_directory(
 data_validacion,
 target_size=(altura, longitud),
 batch_size = batch_size,
 class_mode = 'categorical'
)

Found 810 images belonging to 8 classes.


In [53]:
cnn=Sequential()
cnn.add(Convolution2D(filtrosConv1, tamanio_filtro1, padding='same', input_shape=(altura, longitud, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=tamanio_pool))

cnn.add(Convolution2D(filtrosConv2, tamanio_filtro2, padding='same', activation='relu'))
cnn.add(MaxPooling2D(pool_size=tamanio_pool))

cnn.add(Flatten())
cnn.add(Dense(256, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(clases, activation='softmax'))

In [54]:
cnn.compile(loss='categorical_crossentropy', optimizer=tensorflow.keras.optimizers.Adam(learning_rate=lr), metrics=['accuracy'])

In [55]:
cnn.fit(imagen_entrenamiento, steps_per_epoch=pasos, epochs=epocas, validation_data=imagen_validacion,validation_steps=pasos_validacion)

Epoch 1/10


2023-09-07 23:45:11.307350: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


40/40 [==============================] - ETA: 0s - loss: 5.6486 - accuracy: 0.4748

2023-09-07 23:46:38.446612: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


40/40 [==============================] - 90s 2s/step - loss: 5.6486 - accuracy: 0.4748 - val_loss: 1.0847 - val_accuracy: 0.6250
Epoch 2/10
40/40 [==============================] - 88s 2s/step - loss: 0.9846 - accuracy: 0.6902 - val_loss: 0.7442 - val_accuracy: 0.7312
Epoch 3/10
40/40 [==============================] - 88s 2s/step - loss: 0.7675 - accuracy: 0.7430 - val_loss: 0.6680 - val_accuracy: 0.7500
Epoch 4/10
40/40 [==============================] - 89s 2s/step - loss: 0.6179 - accuracy: 0.7950 - val_loss: 0.6002 - val_accuracy: 0.7563
Epoch 5/10
40/40 [==============================] - 90s 2s/step - loss: 0.5159 - accuracy: 0.8327 - val_loss: 0.4629 - val_accuracy: 0.8250
Epoch 6/10
40/40 [==============================] - 91s 2s/step - loss: 0.5523 - accuracy: 0.8070 - val_loss: 0.5049 - val_accuracy: 0.8313
Epoch 7/10
40/40 [==============================] - 89s 2s/step - loss: 0.3705 - accuracy: 0.8831 - val_loss: 0.4558 - val_accuracy: 0.8375
Epoch 8/10
40/40 [=============

In [56]:
cnn.save('model.h5')
cnn.save_weights('weights.h5')

In [62]:
from skimage.transform import resize
dirname = os.path.join(os.getcwd(), 'DataSet', 'test')
imgpath = dirname + os.sep 
images=[]
# AQUI ESPECIFICAMOS UNAS IMAGENES
filenames = ['DataSet/test/CLASS_07/17-CAPTURE_20220420_174411_517.png',
            'DataSet/test/CLASS_06/14-CAPTURE_20220531_144002_978.png',
            'DataSet/test/CLASS_06/22-CAPTURE_20220421_225004_317.png',
            'DataSet/test/CLASS_05/06-CAPTURE_20220614_064325_983.png',
            'DataSet/test/CLASS_05/13-CAPTURE_20220531_134852_338.png',
            'DataSet/test/CLASS_08/23-CAPTURE_20220421_234229_175.png',
            'DataSet/test/CLASS_02/10-CAPTURE_20220531_103237_465.png']


for filepath in filenames:
    image = plt.imread(filepath,0)
    image_resized = resize(image, (216, 384),anti_aliasing=True,clip=False,preserve_range=True)
    images.append(image_resized)

X = np.array(images) #convierto de lista a numpy
test_X = X.astype('float64')

predicted_classes = cnn.predict(test_X)

for i, img_tagged in enumerate(predicted_classes):
    #print(filenames[i], meat[img_tagged.tolist().index(max(img_tagged))])
    print(filenames[i], np.argmax(img_tagged)+1)

1/1 [==============================] - 0s 184ms/step
DataSet/test/CLASS_07/17-CAPTURE_20220420_174411_517.png 5
DataSet/test/CLASS_06/14-CAPTURE_20220531_144002_978.png 6
DataSet/test/CLASS_06/22-CAPTURE_20220421_225004_317.png 6
DataSet/test/CLASS_05/06-CAPTURE_20220614_064325_983.png 5
DataSet/test/CLASS_05/13-CAPTURE_20220531_134852_338.png 5
DataSet/test/CLASS_08/23-CAPTURE_20220421_234229_175.png 6
DataSet/test/CLASS_02/10-CAPTURE_20220531_103237_465.png 2


# Cargar set de Imágenes

In [ ]:
dirname = os.path.join(os.getcwd(), 'DataSet', 'train')
imgpath = dirname + os.sep 

images = []
directories = []
dircount = []
cant=0

print("leyendo imagenes de ",imgpath)

for root, dirnames, filenames in os.walk(imgpath, topdown=True):
    
    for filename in filenames:
        if re.search("\.(jpg|jpeg|png|bmp|tiff)$", filename):
            cant=cant+1
            filepath = os.path.join(root, filename)
            img_rgb = Image.open(filepath)
            images.append(img_rgb)            
            b = "Leyendo..." + str(cant)
            print (b, end="\r")
                       
    if cant>0:
        print(root, cant)
        directories.append(root)
        dircount.append(cant)
        cant=0

print('Directorios leidos:',len(directories))
print("Imagenes en cada directorio", dircount)
print('suma Total de imagenes en subdirs:',sum(dircount))

In [ ]:
plt.figure(figsize=[15,15])

im = np.asarray(images)
print(im)
print(np.array(images))
#Display the first image in training data
plt.imshow(images[0], cmap='gray', vmin=0, vmax=255)
#plt.title("Ground Truth : {}".format(train_Y[0]))

# Creamos las etiquetas

In [ ]:
labels=[]
indice=0
for cantidad in dircount:
    for i in range(cantidad):
        labels.append(indice)
    indice=indice+1
print("Cantidad etiquetas creadas: ",len(labels))

In [ ]:
meat=[]
indice=0
for directorio in directories:
    name = directorio.split(os.sep)
    print(indice , name[len(name)-1])
    meat.append(name[len(name)-1])
    indice=indice+1

In [ ]:
y = np.array(labels)
X = np.array(images, dtype=np.uint8) #convierto de lista a numpy
#X = np.array(images) #convierto de lista a numpy
#print('X', X)
#print('y',y)
# Find the unique numbers from the train labels
classes = np.unique(y)
nClasses = len(classes)
print('Total number of outputs : ', len(classes))
print('Output classes : ', nClasses)



#Display the first image in training data
#plt.figure(figsize=[5,5])
#plt.imshow(X[0], cmap=plt.get_cmap('gray'))
#plt.title("Imagen")

# Creamos Sets de Entrenamiento y Test

In [ ]:
train_X,test_X,train_Y,test_Y = train_test_split(X,y,test_size=0.3)
print('Training data shape : ', train_X.shape, train_Y.shape)
print('Testing data shape : ', test_X.shape, test_Y.shape)

In [ ]:
plt.figure(figsize=[15,15])

#Display the first image in training data
plt.subplot(1,2,1)
plt.imshow(train_X[0,:,:], cmap='gray', vmin=0, vmax=255)
plt.title("Ground Truth : {}".format(train_Y[0]))

#Display the first image in testing data
plt.subplot(1,2,2)
plt.imshow(test_X[0,:,:], cmap='gray')
plt.title("Ground Truth : {}".format(test_Y[0]))

# Preprocesamos las imagenes

In [ ]:
train_X = train_X.astype('float64')
test_X = test_X.astype('float64')


## Hacemos el One-hot Encoding para la red

In [ ]:
# Change the labels from categorical to one-hot encoding
train_Y_one_hot = to_categorical(train_Y)
test_Y_one_hot = to_categorical(test_Y)

# Display the change for category label using one-hot encoding
print('Original label:', train_Y[0])
print('After conversion to one-hot:', train_Y_one_hot[0])

# Creamos el Set de Entrenamiento y Validación

In [ ]:
#Mezclar todo y crear los grupos de entrenamiento y testing
train_X,valid_X,train_label,valid_label = train_test_split(train_X, train_Y_one_hot, test_size=0.2, random_state=13)

In [ ]:
print(train_X.shape,valid_X.shape,train_label.shape,valid_label.shape)

# Creamos el modelo de CNN

In [ ]:
#declaramos variables con los parámetros de configuración de la red
INIT_LR = 1e-3 # Valor inicial de learning rate. El valor 1e-3 corresponde con 0.001
epochs = 6 # Cantidad de iteraciones completas al conjunto de imagenes de entrenamiento
batch_size = 64 # cantidad de imágenes que se toman a la vez en memoria

In [ ]:
meat_model = Sequential()
meat_model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',padding='same',input_shape=(216,384,3)))
meat_model.add(LeakyReLU(alpha=0.1))
meat_model.add(MaxPooling2D((2, 2),padding='same'))
meat_model.add(Dropout(0.5))

meat_model.add(Flatten())
meat_model.add(Dense(32, activation='linear'))
meat_model.add(LeakyReLU(alpha=0.1))
meat_model.add(Dropout(0.5))
meat_model.add(Dense(nClasses, activation='softmax'))

In [ ]:
meat_model.summary()

In [ ]:
lr_schedule = tensorflow.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=10000,
    decay_rate=0.9)
meat_model.compile(loss=tensorflow.keras.losses.categorical_crossentropy, optimizer=tensorflow.keras.optimizers.Adam(learning_rate=lr_schedule),metrics=['accuracy'])

# Entrenamos el modelo: Aprende a clasificar imágenes

In [ ]:
# este paso puede tomar varios minutos, dependiendo de tu ordenador, cpu y memoria ram libre
# como ejemplo, en mi Macbook pro tarda 4 minutos
meat_train = meat_model.fit(train_X, train_label, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_X, valid_label))

In [ ]:
# guardamos la red, para reutilizarla en el futuro, sin tener que volver a entrenar
meat_model.save("meat_mnist.h5py")

# Evaluamos la red

In [ ]:
test_eval = meat_model.evaluate(test_X, test_Y_one_hot, verbose=1)

In [ ]:
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

In [ ]:
accuracy = meat_train.history['accuracy']
val_accuracy = meat_train.history['val_accuracy']
loss = meat_train.history['loss']
val_loss = meat_train.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
predicted_classes2 = meat_model.predict(test_X)

In [ ]:
predicted_classes=[]
for predicted_meat in predicted_classes2:
    predicted_classes.append(predicted_meat.tolist().index(max(predicted_meat)))
predicted_classes=np.array(predicted_classes)

In [ ]:
predicted_classes.shape, test_Y.shape

# Aprendamos de los errores: Qué mejorar

In [ ]:
correct = np.where(predicted_classes==test_Y)[0]
print("Found %d correct labels" % len(correct))
for i, correct in enumerate(correct[0:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(test_X[correct].reshape(216,384,3), cmap='gray', interpolation='none')
    plt.title("{}, {}".format(meat[predicted_classes[correct]],
                                                    meat[test_Y[correct]]))

    plt.tight_layout()

In [ ]:
incorrect = np.where(predicted_classes!=test_Y)[0]
print("Found %d incorrect labels" % len(incorrect))
for i, incorrect in enumerate(incorrect[0:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(test_X[incorrect].reshape(216,384,3), cmap='gray', interpolation='none')
    plt.title("{}, {}".format(meat[predicted_classes[incorrect]],
                                                    meat[test_Y[incorrect]]))
    plt.tight_layout()

In [ ]:
target_names = ["Class {}".format(i) for i in range(nClasses)]
print(classification_report(test_Y, predicted_classes, target_names=target_names))

# Prediccion de una nueva imagen

In [ ]:
from skimage.transform import resize

images=[]
# AQUI ESPECIFICAMOS UNAS IMAGENES
filenames = ['DataSet/test/CLASS_07/17-CAPTURE_20220420_174411_517.png']

for filepath in filenames:
    image = plt.imread(filepath,0)
    image_resized = resize(image, (216, 384),anti_aliasing=True,clip=False,preserve_range=True)
    images.append(image_resized)

X = np.array(images, dtype=np.uint8) #convierto de lista a numpy
test_X = X.astype('float64')
test_X = test_X

predicted_classes = meat_model.predict(test_X)

for i, img_tagged in enumerate(predicted_classes):
    print(filenames[i], meat[img_tagged.tolist().index(max(img_tagged))])